# Tutorial: Perturbation of the Neoclassical Growth Model

## Advanced Macro: numerical methods

__Warm-up: install and test the `ForwardDiff` library. Check the jacobian function.__

In [1]:
# using ForwardDiff

┌ Info: Precompiling ForwardDiff [f6369f11-7733-5829-9624-2563aa707210]
└ @ Base loading.jl:1278


In [2]:
# import ForwardDiff
# ForwardDiff.jacobian

In [3]:
using ForwardDiff: jacobian

In [17]:
function f(x::Vector{Float64})::Vector{Float64}
    a = x[1]
    b = x[2] + 0.01
    for i=1:10
        b += a*i
    end
    return [(sin(a) + cos(b))^2, 1+(sin(a^2) + cos(b))]
end

f (generic function with 3 methods)

In [18]:
f([0.2, 0.4])

2-element Array{Float64,1}:
 0.36160216439462284
 1.4426536581911549

In [19]:
@time jacobian(f, [0.2, 0.4])

  0.000043 seconds (6 allocations: 560 bytes)


2×2 Array{Float64,2}:
 2.19482  -0.872087
 0.39968  -0.389418

__Warm-up(2): install and test the `NLSolve` library. Find the root of a simple 2 variables function.__


Consider the deterministic neoclassical growth model

- Transition Equation
$$\begin{eqnarray}
k_t & = & (1-\delta) k_{t-1} + i_{t-1} \\\\
z_t & = & \rho z_{t-1}
\end{eqnarray}
$$

- Definition:
$$c_t = \exp(z_t) k_t^\alpha - i_t$$

- Control $i_t\in[-(1-\delta) k_t,k_t^\alpha[$
  - or equivalently $c_t \in [0, k_t^{\alpha}]$

- Objective:
$$\max_{i_t} \sum_{t\geq0} \beta^t U(c_t)$$



- Optimality Condition:
$$\beta  \left[ \frac{\left(c_{t+1}\right)^{-\gamma}}{\left(c_t\right)^{-\gamma}} \left( (1-\delta) + \alpha k_t^{\alpha -1} \right)\right] = 1$$
  - Takes into account the fact that $c_t>0$.
- Definition:
$$c_t = k_t^\alpha - i_t$$


- Calibration:
  - $\beta = 0.96$
  - $\delta = 0.1$
  - $\gamma = 4.0$
  - $\alpha = 0.3$
  - $U(x)=\frac{x^{1-\gamma}}{1-\gamma}$
  

Our goal is to solve for the model dynamics close to the steady-state.

__Create a structure `Calibration` to hold the model parameters.__

In [3]:
struct Calibration
    α::Float64
    β::Float64
    γ::Float64
    δ::Float64
    ρ::Float64
end

In [4]:
c = Calibration(0.3, 0.96, 4.0, 0.1, 0.9)

Calibration(0.3, 0.96, 4.0, 0.1, 0.9)

In [5]:
c.α

0.3

In [6]:
U(x, c) = x^(1-c.γ)/(1-c.γ)

U (generic function with 1 method)

__Define two functions:__
- `transition(z::Float64, k::Float64, i::Float64, p::Calibration)::Tuple{Float64}` which returns productivity and capital at date `t+1` as a function of productivity, capital and investment at date `t`
- `arbitrage(z::Float64, k::Float64, i::Float64, Z::Float64, K::Float64, I::Float64, p::Calibration)::Float64` which returns the residual of the euler equation (lower case variable for date t, upper case for date t+1)


In [7]:
function transition(z::Float64, k::Float64, i::Float64, p::Calibration)
    Z = p.ρ*z
    K = (1-p.δ)*k + i
    return (Z, K)
end

transition (generic function with 1 method)

In [8]:
transition(0.1, 3.0, 0.2, c)

(0.09000000000000001, 2.9000000000000004)

In [11]:
transition(s::Vector, x::Vector, p) = [transition(s[1], s[2], x[1], p)...]

transition (generic function with 2 methods)

In [12]:
transition([0.1, 3.0], [0.2], c)

2-element Array{Float64,1}:
 0.09000000000000001
 2.9000000000000004

In [13]:
using ForwardDiff: jacobian

In [17]:
anonymous = (u->transition(u, [0.2], c))

#5 (generic function with 1 method)

In [20]:
jacobian(anonymous, [0.1, 3.0])

LoadError: [91mMethodError: no method matching transition(::ForwardDiff.Dual{ForwardDiff.Tag{var"#5#6",Float64},Float64,2}, ::ForwardDiff.Dual{ForwardDiff.Tag{var"#5#6",Float64},Float64,2}, ::Float64, ::Calibration)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  transition([91m::Float64[39m, [91m::Float64[39m, ::Float64, ::Calibration) at In[7]:1[39m
[91m[0m  transition([91m::Array{T,1} where T[39m, [91m::Array{T,1} where T[39m, ::Any) at In[11]:1[39m

__Using multiple dispatch, define two variants of the same functions, that take vectors as input and output arguments:__
- `arbitrage(s::Vector{Float64}, x::Vector{Float64}, S::Vector{Float64}, X::Vector{Float64}, p::Calibration)`
- `transition(s::Vector{Float64}, x::Vector{Float64}, p::Calibration)`

__Write a function `steady_state(p::Calibration)::Tuple{Vector,Vector}` which computes the steady-state of the model computed by hand.__ It returns two vectors, one for the states, one for the controls. Check that the steady-state satisfies the model equations.


The first order system satisfies:
$$\begin{eqnarray}A s_t + B x_t + C s_{t+1} + D x_{t+1} & = & 0 \\\\ 
s_{t+1} & = & E s_t + F x_t
 \end{eqnarray}$$

__Define a structure `PerturbedModel` to hold matrices A,B,C,D,E,F.__



__Write a function `first_order_model(s::Vector, x::Vector, p::Calibration)::PerturbedModel`, which returns the first order model, given the steady-state and the calibration. Suggestion: use `ForwardDiff.jl` library.__

__We look for a linear solution $x_t = X s_t$ . Write the matrix equation which `X` must satisfy. Write a function `residual(X::Array, M::PerturbedModel)` which computes the residual of this equation for a given `X`.__


__Write a function `T(X, M::PerturbedModel)`  which implements the time iteration step.__

__Write function `linear_time_iteration(X_0::Matrix, m::PerturbedModel)::Matrix` which implements the time iteration algorithm. Apply it to `X0 = rand(1,2)` and check that the result satisfies the first order model.__

    

__Define two linear operators `L_S(S::Matrix, X_0::Matrix, m::PerturbedModel)::Matrix` and `L_T(S::Matrix, X_0::Matrix, m::PerturbedModel)::Matrix` which implement the derivatives of the simulation and the time-iteration operator respectively.__

__Implement a function `spectral_radius(f::Function)::Float64` which implements the power iteration method to compute the biggest eigenvalues of the two previously defined operators. Check that Blamnchard-Kahn conditions are met.__

__Write a function `simulate(s0::Vector, X::Matrix, p::Calibration, T::Int64)::Tuple{Matrix, Matrix}` to simulate the model over $T$ periods (by using the formula $\Delta s_t = (E + F X) s_{t-11}$. Return a matrix for the states (one line per date) and another matrix for the controls. Bonus: add a keyword option to compute variables levels or log-deviations. If possible, return a DataFrame object.__

__Make some nice plots.__